In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
%matplotlib inline

In [2]:
sub = 'sub-09'
ses = 'ses-01'
battery = 'camcan'
task = 'emotional-memory'

main_dir = '/home/sshankar/ibc/analysis_pipeline/ibc_main/neurospin_data/info/'
datadir = os.path.join(main_dir, sub, battery, ses)
os.chdir(datadir)
csv_files = glob.glob(task + '*.txt')
tsv_files = []
for i in range(len(csv_files)):
    tsv_files.append(task + '_' + sub + '_run-0' + str(i+1) + '.tsv')

In [53]:
# In the log file run start is the file line.
# To extract it read in the first line by itself and then 
# read the rest of the file as a DataFrame.
line1 = pd.read_csv(os.path.join(datadir, csv_files[0]), sep=':', header=None, nrows=1)
run_start = line1[1][0]

In [54]:
file_df = pd.read_csv(os.path.join(datadir, csv_files[0]), header=1, sep=',', skipinitialspace=True)
# file_df = pd.read_csv(os.path.join(datadir, csv_files[0]), sep=',')

In [40]:
run_start

66757.9777

In [41]:
file_df

,img_valence,img_file,img_onset,img_actual_onset,img_onset_end,obj_file,obj_onset,obj_actual_onset,obj_onset_end,resp_key,resp_onset
0,1,\Positive\8200.JPG,66758.7072,66758.7072,66758.7073,\Objects\hose.bmp,66760.7122,66760.7122,66760.7122,y,66762.8257
1,1,\Positive\7430.JPG,66768.8358,66768.8358,66768.8359,\Objects\guitar.bmp,66770.8408,66770.8408,66770.8408,y,66773.4857
2,3,\Negative\9253.JPG,66778.9681,66778.9681,66778.9681,\Objects\glass.bmp,66780.9728,66780.9728,66780.9729,y,66784.0747
3,2,\Neutral\2209.JPG,66789.0997,66789.0997,66789.0998,\Objects\balloon.bmp,66791.1054,66791.1054,66791.1054,y,66793.4211
4,2,\Neutral\7217.JPG,66799.2263,66799.2263,66799.2264,\Objects\highchair.bmp,66801.2314,66801.2314,66801.2315,NaN,-9.0000
5,3,\Negative\9050.JPG,66809.3553,66809.3553,66809.3554,\Objects\folder.bmp,66811.3601,66811.3601,66811.3602,y,66814.3211
6,1,\Positive\8350.JPG,66819.4864,66819.4864,66819.4864,\Objects\kettle.bmp,66821.4908,66821.4908,66821.4908,y,66823.6468
7,2,\Neutral\2514.JPG,66829.6163,66829.6163,66829.6163,\Objects\biscuit.bmp,66831.6210,66831.6210,66831.6211,y,66833.8578
8,3,\Negative\9570.JPG,66839.7443,66839.7443,66839.7443,\Objects\medal.bmp,66841.7487,66841.7487,66841.7487,y,66844.1671
9,3,\Negative\9800.JPG,66849.8734,66849.8734,66849.8734,\Objects\orange.bmp,66851.8777,66851.8777,66851.8777,y,66854.1570


In [55]:
# Find the trial-related rows
# trials = file_df['img_valence'] != 'Scanner'
valence = file_df['img_valence']
negative_trials = file_df['img_valence'].isin(['3'])
neutral_trials = file_df['img_valence'].isin(['2'])
positive_trials = file_df['img_valence'].isin(['1'])

In [43]:
print(len(trials), sum(trials))

60 60


In [44]:
positive_trials

0      True
1      True
2     False
3     False
4     False
5     False
6      True
7     False
8     False
9     False
10     True
11    False
12    False
13     True
14    False
15     True
16    False
17     True
18    False
19    False
20    False
21     True
22    False
23    False
24    False
25     True
26    False
27    False
28    False
29    False
30     True
31    False
32    False
33    False
34     True
35    False
36     True
37    False
38     True
39     True
40    False
41     True
42     True
43    False
44    False
45    False
46     True
47    False
48    False
49     True
50    False
51     True
52     True
53     True
54     True
55    False
56     True
57    False
58    False
59    False
Name: img_valence, dtype: bool

In [45]:
len(file_df)

60

In [56]:
# Trial/Image onsets are in 'img_onset'
img_onsets = file_df['img_onset'].astype(float)

# Object onsets are in 'obj_onset'
obj_onsets = file_df['obj_onset'].astype(float)

# Concatenate the two to get the onsets array
trial_onsets = np.vstack((img_onsets, obj_onsets)).flatten('F')

## Create durations array
# In the task, the image is presented for 8 s. 2 s after image onset, 
# the object is presented, and stays on screen for the remaining 6 s.
img_durations = np.repeat(8,len(file_df))
obj_durations = np.repeat(6,len(file_df))
# Interlace the two durations to create the durations array
durations = (np.vstack((img_durations,obj_durations))).flatten('F')

In [47]:
img_onsets

0     66758.7072
1     66768.8358
2     66778.9681
3     66789.0997
4     66799.2263
5     66809.3553
6     66819.4864
7     66829.6163
8     66839.7443
9     66849.8734
10    66860.0065
11    66870.1383
12    66880.2705
13    66890.4085
14    66900.5444
15    66910.6695
16    66920.8019
17    66930.9365
18    66941.0762
19    66951.2110
20    66961.3467
21    66971.4798
22    66981.6007
23    66991.7338
24    67001.8677
25    67011.9995
26    67022.1334
27    67032.2615
28    67042.3973
29    67052.5223
30    67062.6551
31    67072.7877
32    67082.9148
33    67093.0417
34    67103.1781
35    67113.3115
36    67123.4400
37    67133.5717
38    67143.7148
39    67153.8500
40    67163.9812
41    67174.1191
42    67184.2504
43    67194.3769
44    67204.5092
45    67214.6424
46    67224.7780
47    67234.9104
48    67245.0427
49    67255.1785
50    67265.3115
51    67275.4476
52    67285.5817
53    67295.7174
54    67305.8503
55    67315.9837
56    67326.1230
57    67336.2599
58    67346.39

In [57]:
## Finally, create the trial_types array. This will be an interlacing
## of the 'img_valence' vector for images and 'object' for objects
img_types = np.array(np.ones(len(file_df)), dtype=np.object_)
img_types[negative_trials] = 'negative_image'
img_types[neutral_trials] = 'neutral_image'
img_types[positive_trials] = 'positive_image'

In [58]:
obj_types = np.repeat('object', len(file_df))

In [59]:
trial_types = np.vstack((img_types,obj_types)).flatten('F')

In [60]:
# Create a DataFrame to save to events file
df = pd.DataFrame({'onset':(trial_onsets-run_start), 'duration':durations, 'trial_type':trial_types})
df.to_csv(os.path.join(datadir, tsv_files[0]), sep='\t', float_format='%0.3f', index=False)